In [25]:
import sys
from bs4 import BeautifulSoup
import functools
import requests

In [66]:
def loadElements(inputFile):
    html = open(inputFile).read()
    bs = BeautifulSoup(html)
    elements = bs.select('#anchor_sound_list > div.sound-list._is > ul > li > div.text.lF_')
    return elements

In [67]:
elements = []
for i in range(1, 7+1):
    inputFile = 'page{}.html'.format(i)
    es = loadElements(inputFile)
    elements.extend(es)

In [69]:
def parseElement(x):
    y = next(x.children)
    link = y.attrs['href']
    title = y.text
    item_id = link.split('/')[-1]
    return {'item_id': item_id,'title':title}

In [70]:
items = [parseElement(x) for x in elements]

In [71]:
@functools.lru_cache()
def getDownloadLink(item_id):
    url = 'https://www.ximalaya.com/revision/play/v1/audio?id={}&ptype=1'.format(item_id)
#     print(url)
    r = requests.get(url, headers = {'User-Agent':'curl/7.54.0'})
#     print(r.text, r.status_code)
    link = r.json()['data']['src']
    return link

In [72]:
getDownloadLink('360821293')

'https://aod.cos.tx.xmcdn.com/storages/a177-audiofreehighqps/FC/A1/CMCoOSQDloqnABR3nwBrXce5.m4a'

In [74]:
len(items)

196

In [76]:
len(set([x['title'] for x in items]))

196

In [73]:
for it in items:
    if 'link' not in it:
        link = getDownloadLink(it['item_id'])
        print('OK!!!')
        it['link'] = link
    else:
        print(it)

OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!!
OK!!

In [77]:
with open('download.sh', 'w') as fh:
    for it in items:
        s = """
        if [ ! -f "audio/{title}.m4a" ]; then
            wget "{url}" -O temp.m4a
            mv temp.m4a "audio/{title}.m4a"
        fi
        """.format(title = it['title'], url = it['link'])
        fh.write(s)